## Архитектуры свёрточных сетей

In [1]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time

In [2]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [3]:
def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. " 
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
#torch.backends.cuda.max_split_size_mb = 10000

## DataSet

In [6]:
BATCH_SIZE = 192
transoforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('../data', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('../data', split='balanced', train=False, download=True, transform=transoforms)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

## Предобученные архитектуры

In [7]:
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.MNIST('.', train=True, transform=transoforms, download=True)
test_dataset = tv.datasets.MNIST('.', train=False, transform=transoforms, download=True)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

#### ResNet

In [8]:
model = tv.models.resnet18(pretrained=True)

c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
summary(model.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [11]:
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [12]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [13]:
model.fc = nn.Linear(in_features=512, out_features=10)

In [14]:
print("Params to learn:")
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [15]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [16]:
torch.cuda.empty_cache()

In [17]:
train_loss_resnet18 = train(model, train_iter, test_iter, trainer, 10)

Step 0. time since epoch: 0.458. Train acc: 0.099. Train Loss: 2.436
Step 10. time since epoch: 3.373. Train acc: 0.264. Train Loss: 2.121
Step 20. time since epoch: 6.383. Train acc: 0.455. Train Loss: 1.851
Step 30. time since epoch: 9.295. Train acc: 0.561. Train Loss: 1.639
Step 40. time since epoch: 12.272. Train acc: 0.626. Train Loss: 1.475
Step 50. time since epoch: 15.201. Train acc: 0.671. Train Loss: 1.346
Step 60. time since epoch: 18.116. Train acc: 0.705. Train Loss: 1.235
Step 70. time since epoch: 20.999. Train acc: 0.731. Train Loss: 1.150
Step 80. time since epoch: 23.889. Train acc: 0.752. Train Loss: 1.078
Step 90. time since epoch: 26.803. Train acc: 0.767. Train Loss: 1.016
Step 100. time since epoch: 29.707. Train acc: 0.781. Train Loss: 0.961
Step 110. time since epoch: 32.644. Train acc: 0.793. Train Loss: 0.914
Step 120. time since epoch: 35.551. Train acc: 0.803. Train Loss: 0.873
Step 130. time since epoch: 38.456. Train acc: 0.812. Train Loss: 0.837
Step 14

#### vgg16

In [18]:
# VGG11
model_vgg16 = tv.models.vgg16(pretrained=True)


c:\Users\Анатолий\source\repos\PyTorchtest\PyTorchtest\TorchEnv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
model_vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [20]:
# Убираем требование градиента:
for param in model_vgg16.parameters():
    param.requires_grad = False

In [21]:
model_vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [22]:
model_vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 4096), 
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)

In [23]:
# Распечатать заголовок секции 
print("Params to learn:")  

# Создать пустой список для хранения обучаемых параметров
params_to_update = []  

# Получить именованные параметры модели в виде списка tuple
for name, param in model_vgg16.named_parameters():
    
    # Проверить флаг requires_grad, который указывает нужно ли обучать этот параметр
    if param.requires_grad == True:

        # Добавить параметр в список обучаемых
        params_to_update.append(param)
        
        # Распечатать имя параметра с отступом
        print("\t",name)
        
# Теперь в params_to_update будут только обучаемые параметры модели

Params to learn:
	 classifier.0.weight
	 classifier.0.bias
	 classifier.3.weight
	 classifier.3.bias


In [24]:
trainer = torch.optim.Adam(params_to_update, lr=0.001)

In [25]:
torch.cuda.empty_cache()

In [26]:
train_loss_vgg16 = train(model_vgg16, train_iter, test_iter, trainer, 10)

Step 0. time since epoch: 0.860. Train acc: 0.089. Train Loss: 2.304
Step 10. time since epoch: 49.124. Train acc: 0.718. Train Loss: 1.022
Step 20. time since epoch: 96.388. Train acc: 0.829. Train Loss: 0.623
Step 30. time since epoch: 144.032. Train acc: 0.871. Train Loss: 0.482
Step 40. time since epoch: 191.301. Train acc: 0.896. Train Loss: 0.388
Step 50. time since epoch: 238.515. Train acc: 0.910. Train Loss: 0.332
Step 60. time since epoch: 285.805. Train acc: 0.922. Train Loss: 0.288
Step 70. time since epoch: 333.019. Train acc: 0.930. Train Loss: 0.257
Step 80. time since epoch: 380.258. Train acc: 0.936. Train Loss: 0.234
Step 90. time since epoch: 427.460. Train acc: 0.940. Train Loss: 0.216
Step 100. time since epoch: 474.661. Train acc: 0.944. Train Loss: 0.202
Step 110. time since epoch: 521.867. Train acc: 0.948. Train Loss: 0.187
Step 120. time since epoch: 569.143. Train acc: 0.951. Train Loss: 0.175
Step 130. time since epoch: 616.353. Train acc: 0.954. Train Loss:

In [ ]:
torch.cuda.empty_cache()

### densenet161

In [ ]:
model = tv.models.densenet161(pretrained=True)
# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Linear(in_features=2208, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_loss_densenet161=train(model, train_iter, test_iter, trainer, 10)

### Inception v3 

In [ ]:
BATCH_SIZE = 192
transoforms = tv.transforms.Compose([
    tv.transforms.Grayscale(3),
    tv.transforms.Resize((299, 299)),
    tv.transforms.ToTensor()
])

train_dataset = tv.datasets.EMNIST('.', train=True, transform=transoforms, download=True, split='mnist')
test_dataset = tv.datasets.EMNIST('.', train=False, transform=transoforms, download=True, split='mnist')
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
model = tv.models.inception_v3(pretrained=True)
model = model.to(device)


In [ ]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        y_hat = net(X)
        acc_sum += (y_hat.argmax(axis=1) == y).sum()
        n += y.shape[0]
        return 0
    return acc_sum.item() / n


def train(net, train_iter, test_iter, trainer, num_epochs):
    train_loss = []
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    #net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        net.train()
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat[0], y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat[0].argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 100 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        train_loss.append(train_l_sum / n)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')
    return train_loss

In [ ]:
torch.cuda.empty_cache()

In [ ]:

# Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Linear(in_features=2048, out_features=10)
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)
trainer = torch.optim.Adam(params_to_update, lr=0.001)
train_loss_inception_v3=train(model, train_iter, test_iter, trainer, 10)

In [ ]:
import matplotlib.pyplot as plt

models = ['ResNet18', 'VGG16', 'DenseNet161', 'InceptionV3']
train_losses = [train_loss_resnet18, train_loss_vgg16, train_loss_densenet161, train_loss_inception_v3]

plt.figure(figsize=(10,5))
for i, loss in enumerate(train_losses):
  plt.plot(loss, label=models[i])
plt.xlabel('Epoch')  
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.title('Training Loss Comparison')
plt.show()

print("| Model | Loss |")
print("| - | - |")
for i in range(len(models)):
  print(f"| {models[i]} | {train_losses[i][-1]:.3f} |")